# Demonstration for Ohio Model User's Group

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import Project
from lasso import metcouncil

### Set up the Logger & Parameters

In [2]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

### Input/Output Locations & Parameters

In [3]:
input_dir = os.path.join("C:/", "Users", "david.ory", "documents", "GitHub", "Lasso", "examples", "stpaul")
lasso_dir = os.path.join("C:/", "Users", "david.ory", "documents", "GitHub", "Lasso")
output_dir = input_dir

stpaul_shape_file = os.path.join(input_dir, "shape.geojson")
stpaul_link_file = os.path.join(input_dir, "link.json")
stpaul_node_file = os.path.join(input_dir, "node.geojson")

In [4]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-11 14:51:37, INFO: Lasso base directory set as: C:/Users\david.ory\documents\GitHub\Lasso


### Data Reads

In [5]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

transit_net = TransitNetwork.read(feed_path = input_dir)

2021-03-11 14:51:37, INFO: Reading from following files:
-C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\link.json
-C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\node.geojson
-C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\shape.geojson.
2021-03-11 14:51:52, INFO: Read 66253 links from C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\link.json
2021-03-11 14:51:52, INFO: Read 17159 nodes from C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\node.geojson
2021-03-11 14:51:52, INFO: Read 66253 shapes from C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\shape.geojson
2021-03-11 14:52:13, INFO: Read in transit feed from: C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul
2021-03-11 14:52:13, INFO: Removing calendar.txt from transit network config because file not found
2021-03-11 14:52:13, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-03-11 14:52:13, INFO: Removing fare_attributes

In [6]:
roadway_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

### Make a Scenario 
We manipulate scenarios

In [7]:
base_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net})

2021-03-11 14:52:13, INFO: Creating Scenario


### Make a Cube Network

In [8]:
base_scenario.road_net.links_df = base_scenario.road_net.links_df[base_scenario.road_net.links_df['drive_access'] == 1]

model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(base_scenario.road_net)
model_road_net = metcouncil.calculate_centroidconnect(model_road_net, parameters)
model_road_net.roadway_standard_to_met_council_network()

2021-03-11 14:52:16, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\Lasso
2021-03-11 14:52:16, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-03-11 14:52:16, INFO: Finished calculating centroid connector variable: centroidconnect
2021-03-11 14:52:16, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-03-11 14:52:16, INFO: Didn't detect managed lanes in network.
2021-03-11 14:52:16, INFO: Creating calculated roadway variables.
2021-03-11 14:52:16, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:423: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-11 14:52:20, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2021-03-11 14:52:20, INFO: Adding roadway network variable for county using a spatial join with: C:\Users\david.ory\Documents\GitHub\Lasso\metcouncil_data\county\cb_2017_us_county_5m.shp


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-11 14:52:24, INFO: Finished Calculating county variable: county
2021-03-11 14:52:24, INFO: Calculating MPO as roadway network variable: mpo
2021-03-11 14:52:24, INFO: Finished calculating MPO variable: mpo
2021-03-11 14:52:24, INFO: Adding Counts
2021-03-11 14:52:24, INFO: Adding Variable AADT using Shared Streets Reference from C:\Users\david.ory\Documents\GitHub\Lasso\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2021-03-11 14:52:24, INFO: Added variable: AADT using Shared Streets Reference
2021-03-11 14:52:24, INFO: Adding Variable AADT using Shared Streets Reference from C:\Users\david.ory\Documents\GitHub\Lasso\metcouncil_data\Wisconsin_Lanes_Counts_Median\wi_count_ShSt_API_match.csv
2021-03-11 14:52:24, INFO: Added variable: AADT using Shared Streets Reference
2021-03-11 14:52:24, INFO: Finished adding counts variable: AADT
2021-03-11 14:52:24, INFO: Finished creating ML lanes variable: ML_lanes
2021-03-11 14:52:24, INFO: Finished creating hov corridor variable: se

In [9]:
# model_road_net.write_roadway_as_fixedwidth(
#     output_link_txt = os.path.join(output_dir, "links.txt"),
#    output_node_txt = os.path.join(output_dir, "nodes.txt"),
#    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
#    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
#    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
#)

In [10]:
#standard_transit = StandardTransit.fromTransitNetwork(transit_net)
#standard_transit.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin") )

### Make a Project Card from a Cube Log File

In [11]:
project = Project.create_project(
        base_roadway_network = model_road_net,
        roadway_log_file = os.path.join(lasso_dir, "examples", "stpaul", "cube", "st_paul_example.log"),
        project_name = "Example",
    )

2021-03-11 14:52:29, INFO: No base transit network.
2021-03-11 14:52:29, INFO: No transit changes given or processed.
2021-03-11 14:52:29, INFO: Reading logfile: C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\cube\st_paul_example.log
2021-03-11 14:52:29, INFO: Processed 0 Node lines and 1 Link lines
2021-03-11 14:52:29, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\Lasso
2021-03-11 14:52:29, INFO: Evaluating compatibility between roadway network changes and base network. Not evaluating deletions.
2021-03-11 14:52:29, INFO: Evaluating project changes.
2021-03-11 14:52:29, WARNING: The following attributes are specified in the changes but do not exist in the base network: ['OPERATION_final']
2021-03-11 14:52:29, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\Lasso
2021-03-11 14:52:29, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\Lasso
2021-03-11 14:52:29, INFO: Lasso base directory set as: C:\Users\davi

In [12]:
# project.write_project_card(os.path.join(output_dir, "ohio_mug", "Example.yml"))

2021-03-11 14:52:29, INFO: Wrote project card to: C:/Users\david.ory\documents\GitHub\Lasso\examples\stpaul\ohio_mug\Example.yml


### Apply the Project Card

In [13]:
# build_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net},
#                                          card_directory = os.path.join(lasso_dir, "examples", "stpaul", "ohio_mug"),
#                                          tags = ['checked'],
#                                          validate_project_cards = False)

In [14]:
build_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net})

2021-03-11 14:52:29, INFO: Creating Scenario


In [15]:
build_scenario.apply_project(project.card_data)

2021-03-11 14:52:29, INFO: Applying Example
2021-03-11 14:52:29, INFO: Applying Example
2021-03-11 14:52:29, INFO: Applying Project to Roadway Network: Example


C:\Users\david.ory\anaconda3\envs\lasso_x\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


KeyError: 'change'

In [ ]:
build_scenario.applied_projects

### Write Out Edited Cube Network

In [ ]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(edit_scenario.road_net)
model_road_net.roadway_standard_to_met_council_network()

In [ ]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)